# Introduction
In this workbook, we will build a neural network from scratch.  You will be introduced to PyTorch and the power of neural networks.  We will be working on a synthetic dataset that catalogs the daily number of lemons sold at a lemon stand.  After training, the neural network will be ablet to predict the number of lemons that are likely going to be sold on a given day.  Lets get started!


# Step 1 - Setup our Environment
Lets start by importing the software libraries we will need to build our neural network

### Import PyTorch and Check Version
Here we verify that PyTorch has been loaded and also the version of PyTorch.  Keep in mind, we usually want to run the latest version.  Check at the link below to verify the latest version of PyTorch:
    
    https://pytorch.org/

In [2]:
# Import PyTorch libraries
import torch
from torch import nn 

# Verify PyTorch libraries have been loaded
torch.__version__

'1.7.1'

### Check Our Processing Capability (CPU vs. GPU)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was found for processing'.format(device))

### Download the Dataset
Neural networks need lots of data.  We will start by loading all the lemonade stand data.  Here is will download all the data we have on the lemondade stand.

In [ ]:
!wget https://leaky.ai/wp-content/uploads/2021/03/lemons.csv

# Step 2 - Prep our Data for the Neural Network

The data has been collected in a table with the following columns:  'Weekend', 'Afternoon', 'Sunny', 'Warm', 'Sign', 'CuteDog', 'Ballons', 'Discount', 'FreshlyPicked', 'Ice', 'CleanHouse', 'DogWalked', 'NumberLemonsSold'.  The number we want our neural network to predict is the field called 'NumberLemonsSold'!  In order to train our network, we will need to seperate out our "input" (x) to the neural network and the "output" (y) or value we want it to learn.  Below we take out the last column from the data and place it into "y".

In [17]:
import pandas as pd
df = pd.read_csv('lemons.csv', nrows=1) # Read only the columns of the csv file
columns = df.columns.tolist() # convert the colums to a list
print ("Columns: ", columns)

# Remove the last column from the list
cols_to_use = columns[:len(columns)-1] # drop the last one

# Now build our input (x) and output (y) to the neural network
xDF = pd.read_csv('lemons.csv', usecols=cols_to_use)
yDF = pd.read_csv('lemons.csv', usecols=['NumberLemonsSold'])

# Finally convert our data to PyTorch tensors
x = torch.tensor(xDF.values,dtype=torch.float)
y = torch.tensor(yDF.values,dtype=torch.float)

Columns:  ['Weekend', 'Afternoon', 'Sunny', 'Warm', 'Sign', 'CuteDog', 'Ballons', 'Discount', 'FreshlyPicked', 'Ice', 'CleanHouse', 'DogWalked', 'NumberLemonsSold']


# Step 3 - Build our Neural Network
Below we build a simply neural network that will take in the inputs above (12) and produce a single value as an output.  This network has a single hidden layer of 30 units.

In [5]:
class MyFirstNetwork(nn.Module):
    def __init__(self):
        super(MyFirstNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(12,30),
            nn.ReLU(),
            nn.Linear(30,1)
        )
        
    def forward(self, x):
        logits = self.network(x)
        return logits

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was found for processing'.format(device))

A cuda device was found for processing


In [7]:
model = MyFirstNetwork()

In [8]:
model.to(device)
x = x.to(device)
y = y.to(device)

In [9]:
x

tensor([[1., 1., 1.,  ..., 1., 0., 1.],
        [1., 0., 1.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]], device='cuda:0')

In [10]:
import torch.optim as optim
criterion = torch.nn.MSELoss()  #reduction='sum' maybe?
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(5):
    totalLoss = 0
    for i in range(len(x)):
        # Forward pass: Compute predicted y by passing x to the model
        
        optimizer.zero_grad()
        ypred = model(x[i])
        loss = criterion(ypred, y[i])
        totalLoss+=loss.item()
        loss.backward()
        optimizer.step()
    print (totalLoss)


9662.719689104537
38.79930285994645
14.57657613869361
8.534383775828097
5.9173673910051585


In [11]:
# Data that affects the number of lemons sold in one day
weekend = 1
afternoon = 1
sunny = 1
warm = 0

sign = 1
cutedog = 0
ballons = 1
discount = 1
freshlypicked = 1
ice = 1

# Data that usually does not affect lemons sold
houseclean = 1
dogwentwalk = 1

numlemonssold = 0
if weekend:
    numlemonssold = afternoon + sunny*5 + warm*10 + sign*2 + cutedog + ballons+discount+freshlypicked*5 + ice*2

print ("Actual ", numlemonssold)


Actual  17


In [12]:
model.to('cpu')
x = torch.tensor([weekend, afternoon,sunny,warm, sign, cutedog, 
                 ballons, discount, freshlypicked, ice, houseclean, 
                 dogwentwalk],dtype=float)
y=model(x.float())
print("Model Predicted: ",y.item())

Model Predicted:  17.01375961303711
